# Python implementation of L-Galaxies

This is a playground to test out the possibility of using `python` as an interface into L-Galaxies.

Abbreviations used:
* desc   – descendent
* gal(s) – galax(y|ies)
* prog   – progenitor
* sub(s) – subhalo(s)

List index indentifiers:
* _gid – relative to the graph
* _sid - relative to the snap

– galaxies/orphans do not need an index identifier as they are numpy arrays defined per snap

Ptyhon type identifiers:
* C_ - class
* D_ - numpy dtype
* F_ - function
* b_ - boolean variable
* c_ - constant (value may be set during parameter initialisation)
* [ijk]_ - variable counter (integer)
* n_ - total counter (integer)

In [ ]:
# Imports of generic python routines

%load_ext autoreload
%autoreload 2

import astropy.constants as c
import astropy.units as u
import gc
import h5py
h5py.enable_ipython_completer()
import numpy as np
import sys

# Switch on traceback for warnings
# import traceback
import warnings

def warn_with_traceback(message, category, filename, lineno, file=None, line=None):

    log = file if hasattr(file,'write') else sys.stderr
    traceback.print_stack(file=log)
    log.write(warnings.formatwarning(message, category, filename, lineno, line))

warnings.showwarning = warn_with_traceback

In [ ]:
# Parameters relating to the python code development.
# Parameters relating to the SAM will be set in the input yaml file.

# Location of code
C_DIR='code-C'
PYTHON_DIR='code-python'

# Development limiter
n_GRAPH=np.inf
#n_GRAPH=10       # Change output files to 'test' to avoid over-writing!

# Verbosity
VERBOSITY=1 # 1 - Major program steps only; 1/2 - Major/minor Counters; 3/4/5 - Debugging diags.

# Input files:
# List of all available options
FILE_OPTIONS_LIST='input/available_options.yml'
# List of runtime parameters
FILE_PARAMETERS='input/input.yml'

In [ ]:
# Imports of py-galaxies python routines
sys.path.insert(1,PYTHON_DIR)

# The parameter class, used to store run-time parameters
from parameters import C_parameters

# The graph class, used to store graphs for processing
from graphs import C_graph

# The halo class, used to store halo properties
from halos import C_halo

# The halo_output class and methods used to output halos
from halos import C_halo_output

# The subhalo class, used to store subhalo properties
from subs import C_sub

# The subhalo_output class, used to output subhalos
from subs import C_sub_output

# The galaxy dtype and template
from gals import D_gal, F_gal_template

# The galaxy_output class, used to output galaxies
from gals import C_gal_output

## Functions

Most of the work to be done in external routines, probably to be coded in C for efficiency.

Here we just include the high-level driver routines.

In [ ]:
# These routines will eventually be moved elsewhere.
# For now they are here to help with code development.

def F_process_halos(halos,subs,gals,graph,parameters):
    """
    This is the controlling routine for halo processing.
    Need to think where best to do loop over mini-steps: here or in calling routine (probably the latter)
    Note that all halo, subhalo and gal processing can be done in parallel: not sure how to tell the compiler that.
    """
    # Set flag for existencs (or otherwise) of galaxies.
    if isinstance(gals, np.ndarray):
        b_gals_exist = True
    else:
        b_gals_exist = False
    # Set timestep
    halo=halos[0]
#     dt=(parameters.snap_table['time_in_years'][halo.snap_ID]- \
#         parameters.snap_table['time_in_years'][halo.snap_ID-1]) / \
#         ( parameters.D_param['units']['internal']['time']['Value'] * \
#           eval(parameters.D_param['units']['internal']['time']['Units']).to(u.yr) )
    dt_halo=parameters.dt_halo
    for halo in halos:
        if VERBOSITY>=4: print('Processing halo ',halo.halo_gid)
        if halo.b_done==True:
            raise RuntimeError('halo '+str(halo.halo_gid)+' in graph '+str(halo.graph_ID)+' already processed.')
        # Accretion onto halos.
        # First determine current baryon mass, then call function to set to universal value.
        # The rate could be calculated once at the beginning of a step, but probably very quick
        halo.set_mass_baryon(subs,gals)
        F_halo_set_baryon_fraction(halo,parameters)
        # Reincorporation of ejected gas
        if halo.mass_gas_eject > parameters.mass_minimum_internal: F_halo_reincorporation(halo,parameters)
        # Cooling of gas from halo onto central subhalo (or, in L-Galaxies mode, the most massive subhalo)
        # Cooling occurs only if a central subhalo exists.
        if halo.sub_central_sid != parameters.NO_DATA_INT: 
            sub_central=subs_this_snap[halo.sub_central_sid]
            F_halo_cooling(halo,sub_central,cooling_table,parameters)
            # In l-galaxies mode the virial velocity of the subhalo may have changed, so need to reset that of the central galaxy also
            if parameters.b_lgalaxies:
                if sub_central.gal_central_sid != parameters.NO_DATA_INT:
                    gals_this_snap[sub_central.gal_central_sid]['v_vir']=sub_central.half_mass_virial_speed
        halo.n_dt+=1
        if halo.n_dt==parameters.n_dt_halo: halo.b_done=True
    if subs != None:
        for sub in subs:
            if sub.b_done==True:
                raise RuntimeError('subhalo '+str(sub.sub_gid)+' in graph '+str(sub.graph_ID)+' already processed.')
            if sub.n_gal>1:
                # Initially assume instantaneous merging of galaxies in subhalos
                F_merge_gals(halos[sub.halo_sid],sub,gals[sub.gal_start_sid:sub.gal_end_sid],parameters)
            # Not all subhalos may have hot gas
            if sub.mass_gas_hot > parameters.mass_minimum_internal:
                # debug code
                gal=gals[sub.gal_central_sid]
                if gal['b_exists']==False:
                    print('gal.dtype =',gal.dtype)
                    print('gal =',gal)
                    print('sub.sub_gid, sub.sub_sid =',sub.sub_gid,sub.sub_sid)
                    print('sub.halo_gid, sub.halo_sid =',sub.halo_gid,sub.halo_sid)
                    print('sub.gal_start_sid, sub.gal_end_sid =',sub.gal_start_sid, sub.gal_end_sid)
                    raise AssertionError('Subhalo central galaxy does not exist')
                F_sub_cooling(sub,gal,cooling_table,parameters)
                pass
            sub.n_dt+=1
            if sub.n_dt==parameters.n_dt_halo: sub.b_done=True
    if b_gals_exist:
        # May want to introduce a loop over galaxy timesteps here
        n_dt_gal=int(parameters.dt_halo*1.000001/parameters.timestep_gal_internal)+1
        parameters.n_dt_gal=n_dt_gal
        parameters.dt_gal=parameters.dt_halo/n_dt_gal
        if VERBOSITY >= 2: print('dt_halo, n_dt_gal, dt_gal =',dt_halo, n_dt_gal, parameters.dt_gal)
        for i_dt in range(n_dt_gal):
            for gal in gals:
                if not gal['b_exists']: continue  #  Galaxies may have merged
                if gal['mass_gas_cold'] > parameters.mass_minimum_internal: 
                    mass_stars = F_gal_form_stars(gal,parameters)
                    # If subhalo does not exist, use halo as proxy.  This will work here as only need access to hot gas phase.
                    sub_sid=gal['sub_sid']
                    halo_sid=gal['halo_sid']
                    if sub_sid==parameters.NO_DATA_INT:
                        F_gal_SNR_feedback(mass_stars,gal,halos[halo_sid],halos[halo_sid],parameters)
                        pass
                    else:
                        F_gal_SNR_feedback(mass_stars,gal,subs[sub_sid],halos[halo_sid],parameters)
                        pass
            #F_gal_AGN_feedback(gals_this_snap,subs_this_snap,parameters)

def F_update_halos(halos_last_snap,halos_this_snap,subs_last_snap,subs_this_snap,
                   gals_last_snap,graph,parameters):
    """
    Propagate properties from progenitor halos to descendants.
    Done as a push rather than a pull because sharing determined by progenitor.
    First loop to push halo / subhalo properties; 
    then structured gal array needs to be generated;
    then second push of gal properties.
    """
    # These offsets give the first (sub)halo in this snapshot
    halo_offset=halos_this_snap[0].halo_gid
    if subs_this_snap != None: sub_offset=subs_this_snap[0].sub_gid
    if halos_last_snap != None: halo_offset_last=halos_last_snap[0].halo_gid
    if subs_last_snap != None: sub_offset_last=subs_last_snap[0].sub_gid
    if halos_last_snap != None:
        for halo in halos_last_snap:
            # First determine what fraction to give to each descendant
            desc_start_gid=halo.desc_start_gid
            desc_end_gid=halo.desc_end_gid
            if (halo.n_desc==0): 
                if VERBOSITY >= 4: print('No descendants for halo:',halo,flush=True)
                halo.b_desc_exists = False
                # For now just skip this halo; might want in future to log these occurrences
                # Note that any orphan galaxies will cease to exist, gal['b_exists'] == False
                continue
            fractions=graph.desc_contribution[desc_start_gid:desc_end_gid]/ \
                np.sum(graph.desc_contribution[desc_start_gid:desc_end_gid])
            # The main descendant is the one that inherits the greatest contribution
            desc_main_sid=graph.desc_IDs_gid[desc_start_gid+np.argmax(fractions)]-halo_offset
            halo.desc_main_sid=desc_main_sid
            # assert desc_main_sid < parameters.n_graph
            # All orphans gals go to main descendant so increase relevant orphan count
            halos_this_snap[desc_main_sid].n_orphan+=halo.n_orphan
            # Now loop over descendants transferring properties to them:
            for i_desc in range(desc_start_gid,desc_end_gid):
                desc_halo=halos_this_snap[graph.desc_IDs_gid[i_desc]-halo_offset]
                # assert desc_halo_gid == desc_halo.halo_gid
                if VERBOSITY>=5: print('Processing descendant',desc_halo.halo_gid)
                # Distribute mass to descendants in proportion to fractional contributions
                i_frac=i_desc-desc_start_gid # fraction index corresponding to descendent index i_desc
                desc_halo.mass_from_progenitors+=fractions[i_frac]*halo.mass
                desc_halo.mass_gas_hot+=fractions[i_frac]*halo.mass_gas_hot
                desc_halo.mass_metals_gas_hot+=fractions[i_frac]*halo.mass_metals_gas_hot
                desc_halo.mass_stars+=fractions[i_frac]*halo.mass_stars
                desc_halo.mass_metals_stars+=fractions[i_frac]*halo.mass_metals_stars
            
    # Now loop over the subhalos
    if subs_last_snap != None:
        for sub in subs_last_snap:
            sub_desc_start_gid=sub.desc_start_gid
            sub_desc_end_gid=sub.desc_end_gid
            halo_sid=sub.halo_sid
            desc_main_sid=halos_last_snap[halo_sid].desc_main_sid  # This possibly does not exist
            sub.desc_halo_sid=desc_main_sid
            if sub.n_desc==0:
                # If no descendant, subhalo components get given to the (main descendant of) the host halo
                # and gals become orphans of that halo.   So add to relevant orphan count.
                # Note: it seems that this can result in a huge excess of baryons in the descendant halo.
                halos_this_snap[desc_main_sid].n_orphan+=sub.n_gal
                halos_this_snap[desc_main_sid].mass_gas_hot+=sub.mass_gas_hot
                halos_this_snap[desc_main_sid].mass_metals_gas_hot+=sub.mass_metals_gas_hot
                halos_this_snap[desc_main_sid].mass_stars+=sub.mass_stars
                halos_this_snap[desc_main_sid].mass_metals_stars+=sub.mass_metals_stars
            else:
                # Otherwise the main subhalo descendant gets all the gals and hot gas - 
                # i.e. assume that subhalos cannot split. 
                fractions=graph.sub_desc_contribution[sub_desc_start_gid:sub_desc_end_gid]/ \
                    np.sum(graph.sub_desc_contribution[sub_desc_start_gid:sub_desc_end_gid])
                sub_desc_main_sid=graph.sub_desc_IDs_gid[sub_desc_start_gid+np.argmax(fractions)]-sub_offset
                sub.desc_main_sid=sub_desc_main_sid
                subs_this_snap[sub_desc_main_sid].n_gal+=sub.n_gal
                # So long as subhalos don't split, we can simply inherit the baryon mass
                subs_this_snap[sub_desc_main_sid].mass_baryon+=sub.mass_baryon
                subs_this_snap[sub_desc_main_sid].mass_gas_hot+=sub.mass_gas_hot
                subs_this_snap[sub_desc_main_sid].mass_metals_gas_hot+=sub.mass_metals_gas_hot
                subs_this_snap[sub_desc_main_sid].mass_stars+=sub.mass_stars
                subs_this_snap[sub_desc_main_sid].mass_metals_stars+=sub.mass_metals_stars
                # Now loop over descendants transferring properties to them
                # Only required if we decide that subhalos can split
                # for i_desc in range(sub_desc_start_gid,sub_desc_end_gid):
                #     desc_sub_gid=graph.sub_desc_IDs_gid[i_desc]
                #     desc_sub=subs_this_snap[desc_sub_gid-sub_gid_offset]
                #     assert desc_sub_gid == desc_sub.sub_gid
                #     sub_desc_halo.<quantity>+=fractions[i_desc-desc_start]*sub.<quantity>
        
    # Now count the total number of gals and generate the gal array.
    # This is done as a loop over subhalos within halos so as to keep all gals in a halo 
    # closely associated in the array.
    n_gal=0
    for halo in halos_this_snap:
        n_gal_start=n_gal
        if halo.n_sub>0:
            for sub in subs_this_snap[halo.sub_start_sid:halo.sub_end_sid]:
                # Record the location of this subhalo's gals in the gal lookup table.  This also updates n_gal.
                n_gal=sub.gal_loc(n_gal)
        # Record the starting location of all this halos gals, and of of its orphans, in the gal lookup table, 
        # and update n_gal to include the orphans.
        n_gal=halo.gal_loc(n_gal_start,n_gal)
    if n_gal==0: return None
    # Create new gal array
    # Galaxies initially have b_exist == False and will come into existence if either inherited or 
    # the central galaxy of a subhalo.
    gals_this_snap=np.empty(n_gal,dtype=D_gal)
    gals_this_snap[:]=gal_template
    # Set galaxy gids and update graph galaxy counter (in that order).
    gals_this_snap['gal_gid']=graph.n_gal+np.arange(n_gal)
    graph.n_gal+=n_gal
    
    # Second loop to pass on gal properties.
    if isinstance(gals_last_snap, np.ndarray):
        for halo in halos_last_snap:
            if halo.n_desc == 0: continue
            n_orphan=halo.n_orphan
            if n_orphan > 0:
                # match up orphans
                desc_halo=halos_this_snap[halo.desc_main_sid]
                assert desc_halo != parameters.NO_DATA_INT
                # The is the location of orphan galaxies in the previous snapshot
                gal_last_start_sid=halo.orphan_start_sid
                gal_last_end_sid=gal_last_start_sid+n_orphan
                # and in the current snapshot
                gal_this_start_sid=desc_halo.orphan_count(n_orphan)
                gal_this_end_sid=gal_this_start_sid+n_orphan
                # Copy over all properties
                gals_this_snap[gal_this_start_sid:gal_this_end_sid]=gals_last_snap[gal_last_start_sid:gal_last_end_sid]
                # Update the tracking pointers
                gals_this_snap[gal_this_start_sid:gal_this_end_sid]['halo_gid']=desc_halo.halo_gid
                gals_this_snap[gal_this_start_sid:gal_this_end_sid]['halo_sid']=desc_halo.halo_gid-halo_offset
                gals_this_snap[gal_this_start_sid:gal_this_end_sid]['sub_gid']=parameters.NO_DATA_INT
                gals_this_snap[gal_this_start_sid:gal_this_end_sid]['sub_sid']=parameters.NO_DATA_INT
                # Inherited orphans will not have merged (I think); otherwise the following line could be overwritten
                gals_this_snap[gal_this_start_sid:gal_this_end_sid]['first_prog_gid']=np.arange(gal_this_start_sid,gal_this_end_sid)
                gals_this_snap[gal_this_start_sid:gal_this_end_sid]['next_prog_gid']=parameters.NO_DATA_INT
                #gals_this_snap[gal_this_start_sid:gal_this_end_sid]['v_vir']=desc_halo.half_mass_virial_speed
        if subs_last_snap != None:
            for sub in subs_last_snap:
                n_sub_gal=sub.n_gal
                sub_desc_start_gid=sub.desc_start_gid
                sub_desc_end_gid=sub_desc_start_gid+sub.n_desc
                gal_last_start_sid=sub.gal_start_sid
                gal_last_end_sid=gal_last_start_sid+n_sub_gal
                if sub.n_desc==0:
                    # If no descendant gals become orphans of (the main descendant of) the host halo
                    desc_halo=halos_this_snap[sub.desc_halo_sid]
                    assert desc_halo != parameters.NO_DATA_INT
                    gal_this_start_sid=desc_halo.orphan_count(n_sub_gal)
                    gal_this_end_sid=gal_this_start_sid+n_sub_gal
                    # Copy over all properties
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]=gals_last_snap[gal_last_start_sid:gal_last_end_sid]
                    # Update the tracking pointers
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]['halo_gid']=desc_halo.halo_gid
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]['halo_sid']=desc_halo.halo_gid-halo_offset
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]['sub_gid']=parameters.NO_DATA_INT
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]['sub_sid']=parameters.NO_DATA_INT
                    # New orphans will not have merged (I think); otherwise the following line could be overwritten
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]['first_prog_gid']=np.arange(gal_this_start_sid,gal_this_end_sid)
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]['next_prog_gid']=parameters.NO_DATA_INT
                    #gals_this_snap[gal_this_start_sid:gal_this_end_sid]['v_vir']=desc_halo.half_mass_virial_speed
                else:
                    # Otherwise the main subhalo descendant gets all the gals
                    desc_sub=subs_this_snap[sub.desc_main_sid]
                    desc_halo=halos_this_snap[sub.desc_halo_sid]
                    assert desc_halo != parameters.NO_DATA_INT
                    # Obtain current galaxy counter for this subhalo
                    gal_this_start_sid=desc_sub.gal_count(n_sub_gal)
                    gal_this_end_sid=gal_this_start_sid+n_sub_gal
                    # Copy over all properties
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]=gals_last_snap[gal_last_start_sid:gal_last_end_sid]
                    # Update the tracking pointers
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]['halo_gid']=desc_halo.halo_gid
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]['halo_sid']=desc_halo.halo_gid-halo_offset
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]['sub_gid']=desc_sub.sub_gid
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]['sub_sid']=desc_sub.sub_gid-sub_offset
                    # This is probably wrong: we need to check if there is already an entry for
                    # first_prog_sid for these galaxies and, if so, update next_prog_sid to point to it.
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]['first_prog_gid']=np.arange(gal_this_start_sid,gal_this_end_sid)
                    gals_this_snap[gal_this_start_sid:gal_this_end_sid]['next_prog_gid']=parameters.NO_DATA_INT
                    #gals_this_snap[gal_this_start_sid:gal_this_end_sid]['v_vir']=desc_sub.half_mass_virial_speed

    # Set galaxy properties that are not inherited
    gals_this_snap['graph_ID']=graph.graph_ID
    gals_this_snap['snap_ID']=halos_this_snap[0].snap_ID
    gals_this_snap['gal_sid']=np.arange(len(gals_this_snap))
    # Need to set central galaxies for each subhalo as these are the ones that are accrete gas.
    # Also, newly created galaxies (no progenitors) come into existance at this point.
    for sub in subs_this_snap: 
        F_set_central_galaxy(sub,parameters)    
        gal_central=gals_this_snap[sub.gal_central_sid]
        gal_central['b_exists']=True
        gal_central['sub_gid']=sub.sub_gid
        gal_central['sub_sid']=sub.sub_sid
        gal_central['halo_gid']=sub.halo_gid
        gal_central['halo_sid']=sub.halo_sid
        # The central galaxies have their virial speed updated; the others keep their inherited virial speed
        gals_this_snap[sub.gal_central_sid]['v_vir']=sub.half_mass_virial_speed
     
    # Some sanity checks
    # In principle subhalos should aready have the correct baryon mass: this is a check
    for sub in subs_this_snap:
        if sub.mass_baryon > parameters.mass_minimum_internal:
            if np.abs(sub.mass_baryon/sub.sum_mass_baryon(gals_this_snap)-1.)>1e-4:
                print('gals[''mass_baryon''] =',gals_this_snap[sub.gal_start_sid:sub.gal_end_sid]['mass_baryon'])
                print('sub.mass_gas_hot =',sub.mass_gas_hot)
                print('sub.mass_stars =',sub.mass_stars)
                raise AssertionError('subhalo baryon mass discrepency',
                    sub.graph_ID,sub.snap_ID,sub.sub_sid,sub.mass_baryon,sub.sum_mass_baryon(gals_this_snap))
    # Check that all halos are assigned to a halo
    for gal in gals_this_snap:
        if gal['b_exists'] and gal['halo_sid']==parameters.NO_DATA_INT:
            print(gal.dtype)
            print(gal)
            raise AssertionError('Galaxy not assigned to a halo')
    # Check that all halos are assigned to either a subhalo or a halo
    for gal in gals_this_snap:
        if gal['b_exists'] and gal['sub_sid']==parameters.NO_DATA_INT and gal['halo_sid']==parameters.NO_DATA_INT:
            print(gal.dtype)
            print(gal)
            raise AssertionError('Galaxy assigned neither subhalo nor halo')
    # Check that all galaxies have a finite virial speed
    for gal in gals_this_snap:
        if gal['b_exists'] and gal['v_vir']<1e-10:
            print(gal.dtype)
            print(gal)
            raise AssertionError('Galaxy virial speed is too low')
    return gals_this_snap
    # Check that all galaxies with cold gas have a disc scale length set
    for gal in gals_this_snap:
        if gal['mass_gas_cold']>parameters.mass_minimum_interal and gal['radius_gas_cold']<1e-10:
            print(gal.dtype)
            print(gal)
            raise AssertionError('Galaxy disc scale length not set')
    

def F_update_parameters(graph_file,parameters):
    for key, value in graph_file['Header'].attrs.items():
        if parameters.b_display_parameters: print(key,value)
        exec('parameters.'+key+'=value')
    # At this point, part_mass has been read from the input graph file.  Convert to code units.
    parameters.part_mass=parameters.part_mass*parameters.mass_input_to_internal
    if VERBOSITY >=1:
        print('Internal unit:')
        print('  mass  ',parameters.units_mass_internal.to(c.M_sun))
        print('  length',parameters.units_length_internal)
        print('  time  ',parameters.units_time_internal)
        print('  speed ',parameters.units_speed_internal.to(u.km/u.s))
        print('  temp. ',parameters.units_temperature_internal)
    # Store the total number of graphs in the input file:
    parameters.n_graph=len(graph_file['graph_lengths'])
    # Put code in here to either copy table of snapshot redshifts/times from graph_file,
    # Or calculate them if that does not exist.
    # Currently read from disk:
    parameters.snap_table=np.loadtxt(parameters.snap_file,usecols=[0,2,4],
        dtype=[('snap_ID',np.int32),('redshift',np.float32),('time_in_years',np.float32)])

def F_halo_set_baryon_fraction(halo,parameters):
    """
    Updates the baryon content to be the universal mean, or the sum of the baryon content from
    the progenitors, whichever is larger (so that baryons are not lost).
    Also, baryon mass cannot decrease.
    Any excess baryons arrive in the form of base_metallicity hot gas.
    """  
    delta_baryon=max(0.,parameters.baryon_fraction*max(halo.mass,halo.mass_from_progenitors)-halo.mass_baryon)
    halo.mass_baryon+=delta_baryon
    halo.mass_gas_hot+=delta_baryon
    halo.mass_metals_gas_hot+=delta_baryon*parameters.base_metallicity

def F_halo_reincorporation(halo,parameters):
    """
    Reincorporation of ejected gas.
    Currently just assumes Hen15 model.
    Might be better to pass dt_halo and c_reinc explicitly
    """
    t_reinc = parameters.c_Hen15_reinc/halo.mass
    mass_reinc = halo.mass_gas_eject * (1.-np.exp(-parameters.dt_halo/t_reinc))
    mass_metals_reinc = mass_reinc * (halo.mass_metals_gas_eject/halo.mass_gas_eject)
    halo.mass_gas_eject -= mass_reinc
    halo.mass_metals_gas_eject -= mass_metals_reinc
    halo.mass_gas_hot += mass_reinc
    halo.mass_metals_gas_hot += mass_metals_reinc
    return None
    
def F_set_central_galaxy(sub,parameters):
    """
    Will eventually have fancy code to determine which, if any, galaxy is the central one.
    For now, just make that the first (and only) galaxy.
    """
    sub.gal_central_sid = sub.gal_start_sid
    return None

## Main routine

### Initialisation

In [ ]:
# Read in all the parameters of the run from the yaml and graph input files.

# Read in parameters from yaml input files
parameters=C_parameters(FILE_PARAMETERS,FILE_OPTIONS_LIST)
parameters.verbosity=VERBOSITY

# Import astrophysics modules (could be run-time parameter dependent)
import cooling
cooling_table = cooling.C_cooling(parameters)
from cooling import F_halo as F_halo_cooling
from cooling import F_sub as F_sub_cooling
from star_formation_and_feedback import F_gal_form_stars, F_gal_SNR_feedback
from mergers import F_merge_gals as F_merge_gals
    
# Open graph input file: needs to come before F_update_parameters
graph_file=h5py.File(parameters.graph_file,'r')

# Update parameters with information from graph_file
F_update_parameters(graph_file,parameters)

# Create galaxy template
gal_template=F_gal_template(parameters)

# Create counter to locate graphs within the galaxy output file
n_graph=min(parameters.n_graph,n_GRAPH)
n_gal_graph_start=np.full(n_graph,parameters.NO_DATA_INT,dtype=np.int32)
n_gal=0

# Create output buffers
halo_output=C_halo_output(parameters)
sub_output=C_sub_output(parameters)
gal_output=C_gal_output(parameters)


### Loop over graphs, snapshots, halos, implementing the SAM

In [ ]:
# Loop over graphs
for i_graph in range(n_graph):
    if VERBOSITY >= 1: print('Processing graph',i_graph,flush=True)
    graph = C_graph(i_graph,graph_file,parameters)
    
    # Keep track of location in galaxy output file
    n_gal_graph_start[i_graph]=n_gal
    
    # Loop over snapshots
    halos_last_snap = None
    subs_last_snap = None
    gals_last_snap = None
    for i_snap in graph.snap_ID:
        if i_snap == parameters.NO_DATA_INT: 
            assert halos_last_snap == None
            continue
        if VERBOSITY >= 2: print('Processing snapshot',i_snap,flush=True)
            
        # Initialise halo and subhalo properties.
        # This returns a list of halo and subhalo instances
        # This may be slow: an alternative would be to use np arrays.
        halos_this_snap = [C_halo(i_graph,i_snap,i_halo,graph,parameters) for i_halo in 
                         graph.halo_start_gid[i_snap]+range(graph.n_halo_snap[i_snap])]
        subs_this_snap = None
        if graph.n_sub > 0:
            if graph.n_sub_snap[i_snap] > 0:
                subs_this_snap = [C_sub(i_graph,i_snap,i_sub,graph,parameters) 
                                     for i_sub in graph.sub_start_gid[i_snap]+range(graph.n_sub_snap[i_snap])]
        
        # Propagate information from progenitors to this generation
        # Done as a push rather than a pull because sharing determined by progenitor
        # Have to do this even if no progenitors in order to initialise galaxy array
        gals_this_snap=F_update_halos(halos_last_snap,halos_this_snap,subs_last_snap,
                                          subs_this_snap,gals_last_snap,graph,parameters)
        del halos_last_snap
        del subs_last_snap
        del gals_last_snap
        #gc.collect() # garbage collection -- safe but very slow.

        # Process the halos
        # The determination of timesteps could be done at initialisation (in update_parameters)
        t_step=((parameters.snap_table['time_in_years'][i_snap]- \
            parameters.snap_table['time_in_years'][i_snap-1]) * u.yr / parameters.units_time_internal).value
        n_dt_halo=int(t_step*1.000001/parameters.timestep_halo_internal)+1
        parameters.n_dt_halo=n_dt_halo
        parameters.dt_halo=t_step/n_dt_halo
        if VERBOSITY >= 2: print('t_step, n_dt_halo, dt_halo =',t_step, n_dt_halo,parameters.dt_halo)
        for i_dt in range(n_dt_halo):
            F_process_halos(halos_this_snap,subs_this_snap,gals_this_snap,graph,parameters)
            
        # Once all halos have been done, output results
        # This could instead be done on a halo-by-halo basis in F_process_halos
        halo_output.append(halos_this_snap,parameters)
        if subs_this_snap != None: sub_output.append(subs_this_snap,parameters)
        if isinstance(gals_this_snap, np.ndarray):
            gals_this_snap 
            gal_output.append(gals_this_snap,parameters)
            n_gal+=len(gals_this_snap)
            
        # Rename this_snap data structures to last_snap
        halos_last_snap=halos_this_snap
        subs_last_snap=subs_this_snap
        gals_last_snap=gals_this_snap

        # Delete old references (so that create new objects on next snapshot)
        del halos_this_snap
        del subs_this_snap
        del gals_this_snap

    # Tidy up
    del halos_last_snap
    del subs_last_snap
    del gals_last_snap


###  Tidy up and exit

In [ ]:
# Flush buffers, close files and exit
graph_file.close()
halo_output.close()
sub_output.close()
gal_output.close()

# Reopen galaxy output file to add graph start locations
# Don't simply do this before originally closing the output file, as the close flushes output buffers
gal_output=h5py.File(parameters.galaxy_file,'r+')
# Sanity check that have correct number of galaxies
assert n_gal==len(gal_output['Galaxies'])
# Add graph start locations as new dataset
dset=gal_output.create_dataset('Graph_start_locations',data=n_gal_graph_start,compression='gzip')
# And close
gal_output.close()